In [201]:
import pandas as pd
import re
import collections
import pymorphy2
import nltk
nltk.download("stopwords")

from sklearn.feature_extraction.text import CountVectorizer

morph = pymorphy2.MorphAnalyzer()
stopwords = nltk.corpus.stopwords.words("russian")

[nltk_data] Downloading package stopwords to /home/bun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data = pd.read_csv('Тестовое задание InnoData - Лист1.csv')

In [10]:
data['Review Title'] = data['Review Title'].fillna(' ')
data['text'] = data['Review Title']+' '+data['Review Text']

In [141]:
def to_norm(text):
    clean_text = re.sub('[,.!?a-z0-9-]', '', text.lower()).split()
    for ind, word in enumerate(clean_text):
        try:
            if word in ['не', 'нет']: #and morph.parse(clean_text[ind+1])[0].tag.POS in {'VERB', 'NOUN'}:
                clean_text[ind] = word+'_'+clean_text[ind+1]
                del clean_text[ind+1]
        except IndexError:
            continue
            
    norm_text = [morph.parse(w)[0].normal_form for w in clean_text]
    import_text = [w for w in norm_text if w not in stopwords]
    if len(import_text)<0:
        return 'отзыв_не_обработан'
    else:
        return ' '.join(import_text)

In [142]:
all_reviews = data.dropna(axis=0, how='any')
all_reviews['norm_text'] = all_reviews['text'].apply(to_norm)

all_reviews = all_reviews.reset_index(drop=True)

/home/bun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [216]:
cv = CountVectorizer(ngram_range=(2, 2),max_df=0.95, min_df=0.01, max_features=5000, stop_words = ['отзыв_не_обработан'])
freq = cv.fit_transform(all_reviews['norm_text']).toarray()
voc = cv.get_feature_names()
review_token = pd.concat([all_reviews[['type', 'Star Rating', 'text']], pd.DataFrame(freq, columns=voc)], axis=1)

In [222]:
star_tokens = dict()
review_token1 = review_token[review_token['type']==1]

for name, group in review_token1.groupby('Star Rating'):
    tokens = group.iloc[:,3:].sum().sort_values(ascending=False)[:25]#.reset_index().iloc[:10,0].values
    star_tokens[name] = tokens 

In [223]:
star_tokens

{1: личный кабинет             122
 логин пароль                58
 купить билет                54
 перестать работать          48
 дата рождение               43
 это приложение              37
 приложение не_работать      34
 последний обновление        33
 не_мочь войти               31
 обновление приложение       29
 постоянно вылетать          27
 регистрация рейс            25
 весь равно                  24
 не_мочь зайти               24
 приложение постоянно        22
 войти личный                21
 вход отпечаток              21
 зайти личный                20
 отпечаток палец             19
 приложение работать         19
 приложение весь             18
 выдавать ошибка             16
 зарегистрироваться рейс     16
 покупка билет               16
 приложение вылетать         15
 dtype: int64, 2: личный кабинет             47
 купить билет               30
 логин пароль               28
 последний обновление       24
 это приложение             18
 дата рождение           

In [221]:
review_token.groupby('type')['text'].count()

type
1    3982
2    2000
Name: text, dtype: int64

In [190]:
def find_ngrams(input_list, n):
    return [' '.join(words) for words in zip(*[input_list[i:] for i in range(n)])]

def add_to_dict(diction, key, value):
    if key in diction.keys():
        diction[key].extend(value)
    else:
        diction[key] = value

In [196]:
find_around = dict()

for ind, row in all_reviews.iterrows():
    text = row['norm_text'].split()
    text.extend(find_ngrams(text, 2))
    for i, w in enumerate(text):
        if w in star_tokens[1]:
            try:
                around = text[i-2:i+2]
                del around[2]
            except IndexError:
                pass
            add_to_dict(find_around, w, around)